# FGSD Experiment on REDDIT-MULTI-12K
This notebook runs the Flexible FGSD and Hybrid FGSD experiments on the REDDIT-MULTI-12K dataset.

## Imports

In [11]:
import os
import sys

# Workaround for MKL linking issues (iJIT_NotifyEvent error)
# This must be set before importing torch or numpy
os.environ['MKL_THREADING_LAYER'] = 'GNU'

import torch # Import torch first
import sys

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

print(f"✅ Added to path: {parent_dir}")
print(f"✅ Torch version: {torch.__version__}")

✅ Added to path: /home/stavros/emb3/fgsd_method/src
✅ Torch version: 2.5.1


In [12]:
import sys
import os
import warnings

# Check for PyG
try:
    # torch is already imported in the previous cell
    from torch_geometric.datasets import TUDataset
    from torch_geometric.utils import to_networkx
    HAS_PYG = True
    print("✅ torch_geometric is available. TUDataset will be used.")
except (ImportError, AttributeError) as e:
    print(f"❌ Warning: torch_geometric import failed: {e}")
    print("Will attempt to use memory-efficient manual loading if TUDataset fails.")
    HAS_PYG = False

import numpy as np
import time
import tracemalloc
import gc
import shutil
from dataclasses import dataclass
from typing import List, Tuple, Iterator, Dict, Any, Optional
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import networkx as nx
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

sys.path.append(".")
from fgsd import FlexibleFGSD
from optimized_method import HybridFGSD

❌ Warning: torch_geometric import failed: partially initialized module 'torch_geometric' has no attribute 'typing' (most likely due to a circular import)
Will attempt to use memory-efficient manual loading if TUDataset fails.


## Constants and Helper Functions

In [13]:
# Constants for batch processing
BATCH_SIZE = 500
DATASET_DIR = '/tmp/REDDIT-MULTI-12K'

@dataclass
class GraphRecord:
    """Metadata for a single graph."""
    graph_id: int
    label: int
    node_start: int
    node_end: int

def ensure_dataset_ready():
    """Ensure the dataset is downloaded and extracted."""
    import urllib.request
    import zipfile
    
    os.makedirs(DATASET_DIR, exist_ok=True)
    base_url = 'https://www.chrsmrrs.com/graphkerneldatasets/REDDIT-MULTI-12K.zip'
    zip_path = os.path.join(DATASET_DIR, 'REDDIT-MULTI-12K.zip')
    dataset_path = os.path.join(DATASET_DIR, 'REDDIT-MULTI-12K')
    
    if not os.path.exists(dataset_path):
        print("Downloading REDDIT-MULTI-12K dataset...")
        urllib.request.urlretrieve(base_url, zip_path)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(DATASET_DIR)
        print("Download complete.")
    return dataset_path

def load_metadata(dataset_dir: str) -> List[GraphRecord]:
    """Load graph metadata without loading full graphs into memory."""
    dataset_path = os.path.join(dataset_dir, 'REDDIT-MULTI-12K')
    
    graph_indicator = pd.read_csv(
        os.path.join(dataset_path, 'REDDIT-MULTI-12K_graph_indicator.txt'), 
        header=None
    )[0].values
    graph_labels = pd.read_csv(
        os.path.join(dataset_path, 'REDDIT-MULTI-12K_graph_labels.txt'), 
        header=None
    )[0].values
    
    records = []
    num_graphs = len(graph_labels)
    
    for gid in range(1, num_graphs + 1):
        mask = graph_indicator == gid
        indices = np.where(mask)[0]
        if len(indices) > 0:
            node_start = indices[0]
            node_end = indices[-1] + 1
        else:
            node_start = node_end = 0
        
        records.append(GraphRecord(
            graph_id=gid,
            label=graph_labels[gid - 1] - 1,  # 0-indexed
            node_start=node_start,
            node_end=node_end
        ))
    
    return records

def iter_graph_batches(dataset_dir: str, batch_size: int, records: Optional[List[GraphRecord]] = None) -> Iterator[Tuple[List[nx.Graph], np.ndarray, List[int]]]:
    """
    Iterate over graphs in batches, loading only what's needed.
    Yields: (list of graphs, labels array, graph_ids list)
    """
    dataset_path = os.path.join(dataset_dir, 'REDDIT-MULTI-12K')
    
    if records is None:
        records = load_metadata(dataset_dir)
    
    # Load edges once (unavoidable for graph construction)
    edges_df = pd.read_csv(
        os.path.join(dataset_path, 'REDDIT-MULTI-12K_A.txt'),
        header=None, names=['src', 'dst']
    )
    graph_indicator = pd.read_csv(
        os.path.join(dataset_path, 'REDDIT-MULTI-12K_graph_indicator.txt'),
        header=None
    )[0].values
    
    # Process in batches
    for i in range(0, len(records), batch_size):
        batch_records = records[i:i + batch_size]
        graphs = []
        labels = []
        gids = []
        
        for rec in batch_records:
            G = nx.Graph()
            # Add nodes for this graph
            node_ids = np.where(graph_indicator == rec.graph_id)[0] + 1  # 1-indexed
            for nid in node_ids:
                G.add_node(nid)
            
            # Add edges for this graph
            mask = graph_indicator[edges_df['src'].values - 1] == rec.graph_id
            graph_edges = edges_df[mask][['src', 'dst']].values
            G.add_edges_from(graph_edges)
            
            # Relabel to 0-indexed
            G = nx.convert_node_labels_to_integers(G)
            
            graphs.append(G)
            labels.append(rec.label)
            gids.append(rec.graph_id)
        
        yield graphs, np.array(labels), gids

## Download and Load REDDIT-MULTI-12K Dataset

In [14]:
def download_and_load_reddit():
    """
    Download and load REDDIT-MULTI-12K dataset.
    Prioritizes TUDataset for efficiency, falls back to optimized manual loading.
    """
    # Option 1: Try TUDataset (Preferred)
    if HAS_PYG:
        try:
            root_dir = '/tmp/TUDataset'
            os.makedirs(root_dir, exist_ok=True)
            print(f"Loading REDDIT-MULTI-12K via TUDataset (root={root_dir})...")
            dataset = TUDataset(root=root_dir, name='REDDIT-MULTI-12K')
            
            print(f"Dataset loaded. Converting {len(dataset)} graphs to NetworkX...")
            graphs = []
            labels = []
            for data in dataset:
                g = to_networkx(data, to_undirected=True)
                graphs.append(g)
                labels.append(data.y.item())
            
            labels = np.array(labels)
            if labels.min() > 0: labels = labels - labels.min()
            
            print(f"Successfully loaded {len(graphs)} graphs via TUDataset.")
            return graphs, labels
        except Exception as e:
            print(f"❌ TUDataset loading failed: {e}")
            print("Falling back to manual loading.")
    else:
        print("❌ torch_geometric not available. Skipping TUDataset.")

    # Option 2: Memory-Efficient Manual Loading (Fallback)
    print("Using memory-efficient manual loading...")
    data_dir = '/tmp/REDDIT-MULTI-12K'
    os.makedirs(data_dir, exist_ok=True)
    
    base_url = 'https://www.chrsmrrs.com/graphkerneldatasets/REDDIT-MULTI-12K.zip'
    zip_path = os.path.join(data_dir, 'REDDIT-MULTI-12K.zip')
    
    if not os.path.exists(os.path.join(data_dir, 'REDDIT-MULTI-12K')):
        print("Downloading REDDIT-MULTI-12K dataset...")
        import urllib.request
        import zipfile
        urllib.request.urlretrieve(base_url, zip_path)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
            
    dataset_path = os.path.join(data_dir, 'REDDIT-MULTI-12K')
    
    # Use pandas for memory efficient reading (C engine)
    print("Reading graph indicators...")
    graph_indicator = pd.read_csv(os.path.join(dataset_path, 'REDDIT-MULTI-12K_graph_indicator.txt'), header=None)[0].values
    
    print("Reading graph labels...")
    graph_labels = pd.read_csv(os.path.join(dataset_path, 'REDDIT-MULTI-12K_graph_labels.txt'), header=None)[0].values
    
    num_graphs = len(graph_labels)
    graphs = [nx.Graph() for _ in range(num_graphs)]
    
    # Add nodes
    for node_id, graph_id in enumerate(graph_indicator, start=1):
        graphs[graph_id - 1].add_node(node_id)
        
    print("Reading edges (chunked)...")
    # Read edges in chunks to save RAM
    edge_file = os.path.join(dataset_path, 'REDDIT-MULTI-12K_A.txt')
    chunksize = 100000
    
    for chunk in pd.read_csv(edge_file, header=None, delimiter=',', chunksize=chunksize):
        edges = chunk.values
        # Vectorized lookup: edges[:,0] is u (1-based), so u-1 is index in graph_indicator
        u_indices = edges[:, 0] - 1
        target_graphs = graph_indicator[u_indices]
        
        # Iterate and add
        for i in range(len(edges)):
            u, v = edges[i]
            graph_id = target_graphs[i]
            graphs[graph_id - 1].add_edge(u, v)
            
    # Relabel
    graphs = [nx.convert_node_labels_to_integers(g) for g in graphs]
    labels = graph_labels - 1
    
    return graphs, labels

PREANALYSIS FOR BINS AND RANGE

In [15]:
import matplotlib.pyplot as plt

def compute_spectral_distances_for_func(graphs, func_type, sample_size=500):
    """
    Computes spectral distances based on function f(lambda).
    Uses a random sample of graphs to avoid processing everything for pre-analysis.
    """
    print(f"Computing spectral distances using func='{func_type}' (Sample size={sample_size})...")
    
    # Sample graphs if needed
    np.random.seed(42)
    if len(graphs) > sample_size:
        sample_indices = np.random.choice(len(graphs), sample_size, replace=False)
        sample_graphs = [graphs[i] for i in sample_indices]
    else:
        sample_graphs = graphs
    
    all_distances = []
    node_counts = []

    for G in sample_graphs:
        if G.number_of_nodes() < 2:
            continue
        node_counts.append(G.number_of_nodes())
        try:
            # 1. Normalized Laplacian
            L = np.asarray(nx.normalized_laplacian_matrix(G).todense())
            # 2. Eigen-decomposition
            w, v = np.linalg.eigh(L)
            # 3. Apply function f(lambda)
            if func_type == 'harmonic':
                with np.errstate(divide='ignore', invalid='ignore'):
                    func_w = np.where(w > 1e-9, 1.0 / w, 0)
            elif func_type == 'polynomial':
                func_w = w ** 2
            elif func_type == 'biharmonic':
                with np.errstate(divide='ignore', invalid='ignore'):
                    func_w = np.where(w > 1e-9, 1.0 / (w**2), 0)
            else:
                func_w = w
            
            # 4. Reconstruct f(L)
            fL = v @ np.diag(func_w) @ v.T
            # 5. Compute Distances
            ones = np.ones(L.shape[0])
            S = np.outer(np.diag(fL), ones) + np.outer(ones, np.diag(fL)) - 2 * fL
            S = np.asarray(S)
            # Extract upper triangle distances
            triu_indices = np.triu_indices_from(S, k=1)
            distances = S[triu_indices]
            all_distances.extend(distances.flatten().tolist())
            
        except Exception as e:
            pass

    return np.array(all_distances), np.array(node_counts)

In [ ]:
def analyze_and_visualize(distances, node_counts, func_type):
    """
    Prints statistics and saves the plot.
    """
    save_path = f'reddit_analysis_{func_type}.png'
    print("\n" + "="*60)
    print(f"PRE-ANALYSIS RESULTS FOR: {func_type.upper()}")
    print("="*60)
    # --- RANGE ANALYSIS ---
    min_val = np.min(distances)
    max_val = np.max(distances)
    p95 = np.percentile(distances, 95)
    p99 = np.percentile(distances, 99)
    print(f"1. SPECTRAL DISTANCE VALUES (hist_range)")
    print(f"   Min:  {min_val:.4f}")
    print(f"   Max:  {max_val:.4f}")
    print(f"   95th Percentile: {p95:.4f}")
    print(f"   99th Percentile: {p99:.4f}  <-- RECOMMENDED Range")
    # --- SPARSITY ANALYSIS ---
    print("\n2. SPARSITY CHECK (hist_bins)")
    sim_range = p99
    test_bins = [50, 100, 200, 300, 500]
    print(f"   Assume Range = [0, {sim_range:.2f}]")
    print(f"   {'Bins':<10} | {'Avg Value/Bin':<15} | {'Sparsity Risk'}")
    print("   " + "-"*45)
    for b in test_bins:
        hist, _ = np.histogram(distances, bins=b, range=(0, sim_range))
        avg_hits_per_graph = np.sum(hist) / len(node_counts) / b
        risk = "Low"
        if avg_hits_per_graph < 1.0: risk = "Medium"
        if avg_hits_per_graph < 0.1: risk = "HIGH (Too Sparse)"
        print(f"   {b:<10} | {avg_hits_per_graph:.4f}{'':<10} | {risk}")
    fig, ax = plt.subplots(figsize=(10, 6))
    
    viz_cutoff = np.percentile(distances, 99.5)
    viz_data = distances[distances <= viz_cutoff]
    
    ax.hist(viz_data, bins=100, color='orange', edgecolor='black', alpha=0.7)
    ax.axvline(p95, color='blue', linestyle='--', label=f'95% ({p95:.1f})')
    ax.axvline(p99, color='darkred', linestyle='-', label=f'99% ({p99:.1f})')
    
    ax.set_title(f"REDDIT-MULTI-12K Distribution: {func_type} distances")
    ax.set_xlabel("Spectral Distance Value")
    ax.set_ylabel("Frequency")
    ax.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    print(f"\nPlot saved to: {save_path}")



: 

In [ ]:

# 1. Load Data
graphs, labels = download_and_load_reddit()
print(f"Loaded {len(graphs)} graphs.")

# 2. Define functions to test
functions_to_test = ['harmonic', 'polynomial', 'biharmonic']

# 3. Run Loop
for func in functions_to_test:
    distances, node_counts = compute_spectral_distances_for_func(graphs, func_type=func)
    analyze_and_visualize(distances, node_counts, func_type=func)

❌ torch_geometric not available. Skipping TUDataset.
Using memory-efficient manual loading...
Reading graph indicators...
Reading graph labels...
Reading edges (chunked)...
Loaded 11929 graphs.
Computing spectral distances using func='harmonic' (Sample size=500)...

PRE-ANALYSIS RESULTS FOR: HARMONIC
1. SPECTRAL DISTANCE VALUES (hist_range)
   Min:  0.2500
   Max:  87.6544
   95th Percentile: 7.2961
   99th Percentile: 11.6000  <-- RECOMMENDED Range

2. SPARSITY CHECK (hist_bins)
   Assume Range = [0, 11.60]
   Bins       | Avg Value/Bin   | Sparsity Risk
   ---------------------------------------------
   50         | 2912.5661           | Low
   100        | 1456.2830           | Low
   200        | 728.1415           | Low
   300        | 485.4277           | Low
   500        | 291.2566           | Low

Plot saved to: reddit_analysis_harmonic.png
Computing spectral distances using func='polynomial' (Sample size=500)...

PRE-ANALYSIS RESULTS FOR: POLYNOMIAL
1. SPECTRAL DISTANCE VALU

## Evaluation Function

In [ ]:
def evaluate_classifier(X_train, X_test, y_train, y_test, classifier_name, clf):
    start_time = time.time()
    clf.fit(X_train, y_train)
    train_time = time.time() - start_time

    y_train_pred = clf.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)

    start_time = time.time()
    y_pred = clf.predict(X_test)
    inference_time = time.time() - start_time

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    try:
        y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
        if hasattr(clf, 'predict_proba'):
            y_score = clf.predict_proba(X_test)
        elif hasattr(clf, 'decision_function'):
            y_score = clf.decision_function(X_test)
            if len(y_score.shape) == 1:
                y_score = y_score.reshape(-1, 1)
        else:
            y_score = None

        if y_score is not None and y_test_bin.shape[1] > 1:
            auc = roc_auc_score(y_test_bin, y_score, average='weighted', multi_class='ovr')
        else:
            auc = None
    except:
        auc = None

    return {
        'classifier': classifier_name,
        'train_accuracy': train_accuracy,
        'accuracy': accuracy,
        'f1_score': f1,
        'auc': auc,
        'train_time': train_time,
        'inference_time': inference_time
    }

## Run FGSD Experiments

In [ ]:
def run_experiment(configs, test_size=0.15, random_state=42):
    print("Ensuring dataset is ready on disk...")
    ensure_dataset_ready()
    
    records = load_metadata(DATASET_DIR)
    all_labels = np.array([r.label for r in records])
    all_gids = np.array([r.graph_id for r in records])
    
    # Pre-calculate Stratified Split Indices
    # We split indices, not graphs.
    indices = np.arange(len(records))
    train_idx, test_idx = train_test_split(
        indices, test_size=test_size, random_state=random_state, stratify=all_labels
    )
    
    # Create mask for fast checking
    is_train = np.zeros(len(records), dtype=bool)
    is_train[train_idx] = True
    
    results = []

    for i, config in enumerate(configs):
        func = config['func']
        
        print(f"\nExperiment {i+1}/{len(configs)}: {func}")
        print(f"Generating embeddings batch-wise to save RAM...")

        start_time = time.time()
        
        # We need to compute embeddings for ALL graphs.
        # We can't reuse one model instance easily if the model stores state during fit.
        # However, FGSD/HybridFGSD usually don't learn parameters from other graphs (it's a histogram).
        # We will instantiate the model and process batch by batch.
        
        if func == 'hybrid':
            model = HybridFGSD(
                harm_bins=config.get('harm_bins', 200), harm_range=config.get('harm_range', 20),
                pol_bins=config.get('pol_bins', 70), pol_range=config.get('pol_range', 4.1),
                func_type='hybrid', seed=random_state
            )
        else:
            model = FlexibleFGSD(hist_bins=config['bins'], hist_range=config['range'], func_type=func, seed=random_state)

        # Storage for embeddings
        # We don't know embedding size until we run first graph, but usually it's bins or bins*2
        embeddings_list = []
        
        for graphs, labels, _ in tqdm(iter_graph_batches(DATASET_DIR, BATCH_SIZE), desc="Embedding"):
            # Compute embeddings for this batch
            # We fit the model on the batch. Since FGSD is instance-based, this is valid.
            model.fit(graphs)
            batch_emb = model.get_embedding() # numpy array [batch, dim]
            embeddings_list.append(batch_emb)
            
            del graphs
            gc.collect()
            
        # Concatenate all embeddings
        X_all = np.vstack(embeddings_list)
        y_all = all_labels # aligns with records
        
        generation_time = time.time() - start_time
        print(f"Embedding Complete. Shape: {X_all.shape}")

        # Split based on pre-calculated indices
        X_train = X_all[train_idx]
        y_train = y_all[train_idx]
        X_test = X_all[test_idx]
        y_test = y_all[test_idx]

        classifiers = {
            'SVM (RBF) + Scaler': make_pipeline(
                StandardScaler(),
                SVC(kernel='rbf', C=100, gamma='scale', probability=True, random_state=random_state)
            ),
            'Random Forest': RandomForestClassifier(n_estimators=1000, random_state=random_state, n_jobs=-1),
            'MLP': make_pipeline(
                StandardScaler(), 
                MLPClassifier(hidden_layer_sizes=(1024, 512, 256, 128), max_iter=1000, early_stopping=True, random_state=random_state)
            )
        }

        for clf_name, clf in classifiers.items():
            res = evaluate_classifier(X_train, X_test, y_train, y_test, clf_name, clf)
            res.update(config)
            res['generation_time'] = generation_time
            results.append(res)
            print(f"  -> {clf_name}: Test Acc={res['accuracy']:.4f}")

    return results

## Summary Function

In [ ]:
def print_summary(results):
    print("\n" + "="*120)
    print("SUMMARY OF RESULTS")
    print("="*120)
    print(f"{'Func':<12} {'Parameters':<30} {'Classifier':<20} {'Train Acc':<11} {'Test Acc':<10} {'F1':<10} {'GenTime':<8}")
    print("-" * 120)

    sorted_results = sorted(results, key=lambda x: x['accuracy'], reverse=True)

    for r in sorted_results:
        if r['func'] == 'hybrid':
            params = f"h_bins={r.get('harm_bins')},h_range={r.get('harm_range')},p_bins={r.get('pol_bins')},p_range={r.get('pol_range')}"
        else:
            params = f"bins={r.get('bins')}, range={r.get('range')}"

        print(f"{r['func']:<12} {params:<30} {r['classifier']:<20} "
              f"{r['train_accuracy']:<11.4f} {r['accuracy']:<10.4f} {r['f1_score']:<10.4f} {r['generation_time']:<8.2f}")

## Run the Full Experiment

In [ ]:
configs = [
    {'func': 'hybrid', 'harm_bins': 100, 'harm_range': 3.5, 'pol_bins': 200, 'pol_range': 3.5},
    {'func': 'polynomial', 'bins': 200, 'range': 3.1},
    {'func': 'harmonic', 'bins': 100, 'range': 3.5},
]

print("Starting Multi-Configuration FGSD Experiment on REDDIT-MULTI-12K...")
results = run_experiment(configs)
print_summary(results)

df = pd.DataFrame(results)
df.to_csv("fgsd_reddit_results.csv", index=False)

Starting Multi-Configuration FGSD Experiment on REDDIT-MULTI-12K...
Loading REDDIT-MULTI-12K dataset...


Starting Multi-Configuration FGSD Experiment on REDDIT-MULTI-12K...
Loading REDDIT-MULTI-12K dataset...


KeyboardInterrupt: 